In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

In [2]:
def stratified_group_k_fold(X, y, groups, k, seed=None): # 원본 코드
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [3]:

from pycocotools.coco import COCO

coco = COCO('/opt/ml/detection/dataset/train.json')


loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [4]:
df = pd.DataFrame(coco.dataset['annotations'])
X = df['id']
y = df['category_id']
groups = df['image_id']
seed = 777
k = 5


In [5]:

def get_distribution(y_vals):
        y_distr = Counter(y_vals)
        y_vals_sum = sum(y_distr.values())
        return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [6]:
distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(X, y, groups, k)):
    dev_y, val_y = y[dev_ind], y[val_ind]
    dev_groups, val_groups = groups[dev_ind], groups[val_ind]
    
    assert len(set(dev_groups) & set(val_groups)) == 0
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - fold {fold_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - fold {fold_ind}')

print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(y) + 1)])

Distribution per class:


,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
development set - fold 0,17.14%,27.45%,3.87%,4.04%,4.24%,12.72%,5.46%,22.38%,0.69%,2.02%
validation set - fold 0,17.13%,27.43%,3.88%,4.06%,4.25%,12.71%,5.46%,22.36%,0.69%,2.03%
development set - fold 1,17.14%,27.45%,3.87%,4.05%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 1,17.12%,27.44%,3.89%,4.04%,4.25%,12.72%,5.46%,22.37%,0.69%,2.03%
development set - fold 2,17.14%,27.45%,3.88%,4.05%,4.25%,12.71%,5.46%,22.37%,0.69%,2.02%
validation set - fold 2,17.13%,27.44%,3.87%,4.04%,4.23%,12.72%,5.47%,22.38%,0.69%,2.03%
development set - fold 3,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 3,17.15%,27.46%,3.87%,4.04%,4.24%,12.71%,5.45%,22.38%,0.69%,2.01%
development set - fold 4,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
